In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_fscore_support

## Adam Experiment

In [ ]:
# ==========================================
# 1. PERSIAPAN DATA (CONFIG 3: 70:30)
# ==========================================
print("📂 Loading Data...")
df = pd.read_csv('../data/processed_attrition_data.csv') 
X = df.drop(columns=['Attrition_Risk_Level'])
y = df['Attrition_Risk_Level']

# ⚠️ PERUBAHAN DISINI UNTUK CONFIG 3 (70:30)
print("⚙️ Setting Split Data ke 70:30 (Test Size = 0.3)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y  # <--- GANTI JADI 0.3
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ==========================================
# 2. VARIABEL EKSPERIMEN (GRID SEARCH)
# ==========================================
list_lr = [0.04, 0.08, 0.06, 0.005, 0.009, 0.007, 0.2, 0.6, 0.012, 0.010, 0.015, 0.018, 0.0111]
list_activation = ['logistic', 'relu', 'tanh']
list_architecture = [(100, 50), (32, 64, 16), (128, 64, 32, 16), (256, 128, 64, 32, 16)]

results = []
counter = 1
total_exp = len(list_lr) * len(list_activation) * len(list_architecture)

print(f"🚀 MEMULAI {total_exp} KOMBINASI EKSPERIMEN (CONFIG 3: 70:30)...")
print("=" * 60)

# ==========================================
# 3. TRIPLE LOOPING
# ==========================================
for arch in list_architecture:
    for act in list_activation:
        for LR in list_lr:
            
            # Definisi Model
            mlp = MLPClassifier(
                hidden_layer_sizes=arch,
                activation=act,
                solver='sgd',
                learning_rate_init=LR,
                momentum=0.8,
                max_iter=375,
                random_state=52,
                early_stopping=True,
                validation_fraction=0.1,
                verbose=False
            )
            
            # Training
            mlp.fit(X_train_scaled, y_train)
            
            # Evaluasi
            y_train_pred = mlp.predict(X_train_scaled)
            mse_train = mean_squared_error(y_train, y_train_pred)
            acc_train = accuracy_score(y_train, y_train_pred)
            
            # Hitung Detail Per Kelas
            precision, recall, f1, support = precision_recall_fscore_support(
                y_train, y_train_pred, labels=[0, 1, 2], zero_division=0
            )
            
            # Simpan Data
            row = {
                "No": counter,
                "Split": "70:30",  # <--- Label diganti jadi 70:30
                "Arsitektur": str(arch),
                "Activation": act,
                "Learning Rate": LR,
                "Epoch": mlp.n_iter_,
                "Loss (MSE)": round(mse_train, 5),
                "Acc Train": round(acc_train, 4),
                
                # --- Detail Metrics ---
                "Train Precision 0": round(precision[0], 4),
                "Train Precision 1": round(precision[1], 4),
                "Train Precision 2": round(precision[2], 4),
                
                "Train Recall 0": round(recall[0], 4),
                "Train Recall 1": round(recall[1], 4),
                "Train Recall 2": round(recall[2], 4),
                
                "Train F1 0": round(f1[0], 4),
                "Train F1 1": round(f1[1], 4),
                "Train F1 2": round(f1[2], 4),
                
                "Support 0": support[0],
                "Support 1": support[1],
                "Support 2": support[2]
            }
            results.append(row)
            print(f"✅ Exp {counter}: {act} | LR {LR} | Acc: {acc_train:.2%}")
            counter += 1

# ==========================================
# 4. EXPORT
# ==========================================
df_results = pd.DataFrame(results)

# Nama file dibedakan
nama_file = "Hasil_GridSearch_Config3_70_30.csv" 

df_results.to_csv(nama_file, index=False)
print("\n" + "="*60)
print(f"🎉 SELESAI! File tersimpan sebagai: {nama_file}")
print("Silakan copy isi file ini ke Sheet Config-3 di Spreadsheet.")

📂 Loading Data...
⚙️ Setting Split Data ke 70:30 (Test Size = 0.3)...
🚀 MEMULAI 156 KOMBINASI EKSPERIMEN (CONFIG 3: 70:30)...
✅ Exp 1: logistic | LR 0.04 | Acc: 99.32%
✅ Exp 2: logistic | LR 0.08 | Acc: 98.96%
✅ Exp 3: logistic | LR 0.06 | Acc: 99.01%
✅ Exp 4: logistic | LR 0.005 | Acc: 98.86%
✅ Exp 5: logistic | LR 0.009 | Acc: 99.25%
✅ Exp 6: logistic | LR 0.007 | Acc: 99.33%
✅ Exp 7: logistic | LR 0.2 | Acc: 98.16%
✅ Exp 8: logistic | LR 0.6 | Acc: 92.11%
✅ Exp 9: logistic | LR 0.012 | Acc: 99.45%
✅ Exp 10: logistic | LR 0.01 | Acc: 99.44%
✅ Exp 11: logistic | LR 0.015 | Acc: 99.44%
✅ Exp 12: logistic | LR 0.018 | Acc: 98.94%
✅ Exp 13: logistic | LR 0.0111 | Acc: 99.44%
✅ Exp 14: relu | LR 0.04 | Acc: 99.05%
✅ Exp 15: relu | LR 0.08 | Acc: 99.20%
✅ Exp 16: relu | LR 0.06 | Acc: 98.57%
✅ Exp 17: relu | LR 0.005 | Acc: 98.89%
✅ Exp 18: relu | LR 0.009 | Acc: 98.41%
✅ Exp 19: relu | LR 0.007 | Acc: 99.01%
✅ Exp 20: relu | LR 0.2 | Acc: 98.50%
✅ Exp 21: relu | LR 0.6 | Acc: 86.33%
✅ Exp

## SGD Experiment

In [6]:
# ==========================================
# 1. PERSIAPAN DATA (CONFIG 3: 70:30)
# ==========================================
print("📂 Loading Data...")
# Pastikan path '../data/' ini benar sesuai folder laptopmu
# Kalau error file not found, ganti jadi 'processed_attrition_data.csv' saja
df = pd.read_csv('../data/processed_attrition_data.csv') 
X = df.drop(columns=['Attrition_Risk_Level'])
y = df['Attrition_Risk_Level']

# ⚠️ SETTING SPLIT 70:30
print("⚙️ Setting Split Data ke 70:30 (Test Size = 0.3)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y 
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ==========================================
# 2. VARIABEL EKSPERIMEN (SGD)
# ==========================================
list_lr = [0.04, 0.08, 0.06, 0.005, 0.009, 0.007, 0.2, 0.6, 0.012, 0.010, 0.015, 0.018, 0.0111]
list_activation = ['logistic', 'relu', 'tanh']
list_architecture = [(100, 50), (32, 64, 16), (128, 64, 32, 16), (256, 128, 64, 32, 16)]

results = []
counter = 1
total_exp = len(list_lr) * len(list_activation) * len(list_architecture)

print(f"🚀 MEMULAI {total_exp} KOMBINASI EKSPERIMEN SGD (CONFIG 3: 70:30)...")
print("=" * 60)

# ==========================================
# 3. TRIPLE LOOPING
# ==========================================
for arch in list_architecture:
    for act in list_activation:
        for LR in list_lr:
            
            # Definisi Model SGD
            mlp = MLPClassifier(
                hidden_layer_sizes=arch,
                activation=act,
                solver='sgd',             # <--- SGD
                learning_rate_init=LR,
                momentum=0.8,             # <--- Momentum Aktif
                max_iter=375,
                random_state=52,
                early_stopping=True,
                validation_fraction=0.1,
                verbose=False
            )
            
            # Training
            mlp.fit(X_train_scaled, y_train)
            
            # Evaluasi
            y_train_pred = mlp.predict(X_train_scaled)
            mse_train = mean_squared_error(y_train, y_train_pred)
            acc_train = accuracy_score(y_train, y_train_pred)
            
            # Hitung Detail Per Kelas
            precision, recall, f1, support = precision_recall_fscore_support(
                y_train, y_train_pred, labels=[0, 1, 2], zero_division=0
            )
            
            # Simpan Data
            row = {
                "No": counter,
                "Split": "70:30",
                "Arsitektur": str(arch),
                "Activation": act,
                "Learning Rate": LR,
                "Epoch": mlp.n_iter_,
                "Loss (MSE)": round(mse_train, 5),
                "Acc Train": round(acc_train, 4),
                
                # --- Detail Metrics ---
                "Train Precision 0": round(precision[0], 4),
                "Train Precision 1": round(precision[1], 4),
                "Train Precision 2": round(precision[2], 4),
                
                "Train Recall 0": round(recall[0], 4),
                "Train Recall 1": round(recall[1], 4),
                "Train Recall 2": round(recall[2], 4),
                
                "Train F1 0": round(f1[0], 4),
                "Train F1 1": round(f1[1], 4),
                "Train F1 2": round(f1[2], 4),
                
                "Support 0": support[0],
                "Support 1": support[1],
                "Support 2": support[2]
            }
            results.append(row)
            print(f"✅ Exp {counter}: SGD | {act} | LR {LR} | Acc: {acc_train:.2%}")
            counter += 1

# ==========================================
# 4. EXPORT
# ==========================================
df_results = pd.DataFrame(results)

# ⚠️ NAMA FILE DIBEDAKAN (Kasih label SGD)
nama_file = "Hasil_GridSearch_SGD_7030.csv" 

df_results.to_csv(nama_file, index=False)
print("\n" + "="*60)
print(f"🎉 SELESAI! File tersimpan sebagai: {nama_file}")
print("Silakan copy isi file ini ke bagian bawah Sheet Config-3.")

📂 Loading Data...
⚙️ Setting Split Data ke 70:30 (Test Size = 0.3)...
🚀 MEMULAI 156 KOMBINASI EKSPERIMEN SGD (CONFIG 3: 70:30)...
✅ Exp 1: SGD | logistic | LR 0.04 | Acc: 99.39%
✅ Exp 2: SGD | logistic | LR 0.08 | Acc: 99.62%
✅ Exp 3: SGD | logistic | LR 0.06 | Acc: 99.40%
✅ Exp 4: SGD | logistic | LR 0.005 | Acc: 50.00%
✅ Exp 5: SGD | logistic | LR 0.009 | Acc: 50.00%
✅ Exp 6: SGD | logistic | LR 0.007 | Acc: 50.00%
✅ Exp 7: SGD | logistic | LR 0.2 | Acc: 99.00%
✅ Exp 8: SGD | logistic | LR 0.6 | Acc: 98.62%
✅ Exp 9: SGD | logistic | LR 0.012 | Acc: 99.10%
✅ Exp 10: SGD | logistic | LR 0.01 | Acc: 98.57%
✅ Exp 11: SGD | logistic | LR 0.015 | Acc: 99.37%
✅ Exp 12: SGD | logistic | LR 0.018 | Acc: 98.90%
✅ Exp 13: SGD | logistic | LR 0.0111 | Acc: 99.15%
✅ Exp 14: SGD | relu | LR 0.04 | Acc: 99.17%
✅ Exp 15: SGD | relu | LR 0.08 | Acc: 99.15%
✅ Exp 16: SGD | relu | LR 0.06 | Acc: 99.56%
✅ Exp 17: SGD | relu | LR 0.005 | Acc: 99.32%
✅ Exp 18: SGD | relu | LR 0.009 | Acc: 99.21%
✅ Exp 19:

In [3]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report

# ==========================================
# KONFIGURASI 3: 70:30 (SGD) - FINAL REPORT
# ==========================================

# 1. LOAD DATA
# Pastikan path file sesuai dengan lokasi di laptop Anda
try:
    df = pd.read_csv('../data/processed_attrition_data.csv')
except FileNotFoundError:
    df = pd.read_csv('processed_attrition_data.csv') # Fallback jika file di folder yang sama

X = df.drop('Attrition_Risk_Level', axis=1)
y = df['Attrition_Risk_Level']

# 2. SETTING PARAMETER (Sesuai Model Terbaik Config-3)
config = {
    'name': 'Config-3 (70:30)',
    'test_size': 0.30,  # 30% Data Test
    'params': {
        'hidden_layer_sizes': (256, 128, 64, 32, 16), # Arsitektur 5 Layer
        'activation': 'relu',
        'solver': 'sgd',          # Solver SGD
        'learning_rate_init': 0.1111, # LR Unik Anda
        'max_iter': 18,           # Epoch 18
        'random_state': 52        # Konsisten dengan spreadsheet
    }
}

print(f"=== PENGUJIAN DETAIL: {config['name']} ===")

# 3. SPLIT DATA
# random_state=42 pada split agar pembagian data konsisten
indices = np.arange(len(X))
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=config['test_size'], random_state=42, stratify=y
)

# 4. SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. TRAINING
model = MLPClassifier(**config['params'])
model.fit(X_train_scaled, y_train)

# 6. PREDIKSI
y_pred = model.predict(X_test_scaled)
proba = model.predict_proba(X_test_scaled) # Untuk Confidence level

# =========================================================
# BAGIAN A: OUTPUT TABEL METRIK (Untuk Sheet Bagian II)
# =========================================================
report = classification_report(y_test, y_pred, output_dict=True)
acc = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

laporan_dict = {
    'No': [1],
    'Split Data Configuration': ['70:30'],
    'MLP Architecture': [str(config['params']['hidden_layer_sizes'])],
    'Learning Rate': [config['params']['learning_rate_init']],
    'Solver': [config['params']['solver']],
    'Momentum': [0.9],
    'Activation': [config['params']['activation']],
    'Loss Test (MSE)': [round(mse, 5)],
    'Acc Test': [round(acc, 4)],
    
    # --- PRECISION ---
    'Prec_Class_0': [round(report['0']['precision'], 4)],
    'Prec_Class_1': [round(report['1']['precision'], 4)],
    'Prec_Class_2': [round(report['2']['precision'], 4)],
    'Prec_Avg':     [round(report['weighted avg']['precision'], 4)],
    
    # --- RECALL ---
    'Rec_Class_0': [round(report['0']['recall'], 4)],
    'Rec_Class_1': [round(report['1']['recall'], 4)],
    'Rec_Class_2': [round(report['2']['recall'], 4)],
    'Rec_Avg':     [round(report['weighted avg']['recall'], 4)],
    
    # --- F1 SCORE ---
    'F1_Class_0': [round(report['0']['f1-score'], 4)],
    'F1_Class_1': [round(report['1']['f1-score'], 4)],
    'F1_Class_2': [round(report['2']['f1-score'], 4)],
    'F1_Avg':     [round(report['weighted avg']['f1-score'], 4)],
    
    # --- SUPPORT ---
    'Supp_Class_0': [report['0']['support']],
    'Supp_Class_1': [report['1']['support']],
    'Supp_Class_2': [report['2']['support']],
    'Supp_Total':   [report['macro avg']['support']],
    
    'Random State': [config['params']['random_state']],
    'Validation Fraction': [0.1]
}

df_laporan = pd.DataFrame(laporan_dict)

print("\n>>> DATA METRIK (Copy ke Sheet 'Config-3 (70:30) Test'):")
pd.set_option('display.max_columns', None)
print(df_laporan)

# =========================================================
# BAGIAN B: OUTPUT ANALISIS ERROR (Untuk Sheet Bagian III)
# =========================================================

# Ambil fitur asli data test
df_analisis_full = X.loc[idx_test].copy()

# Masukkan hasil prediksi
df_analisis_full.insert(0, 'Status', np.where(y_test == y_pred, 'Benar', 'SALAH'))
df_analisis_full.insert(1, 'Prediksi_MLP', y_pred)
df_analisis_full.insert(2, 'Label_Asli', y_test.values)
df_analisis_full.insert(3, 'Confidence', np.round(np.max(proba, axis=1), 4))

# Simpan ke CSV
nama_file_csv = 'Analisis_Full_4500_Data_Config3.csv'
df_analisis_full.to_csv(nama_file_csv)

print(f"\n>>> ANALISIS ERROR:")
print(f"Total Data Test : {len(y_test)}")
print(f"Total Salah     : {len(df_analisis_full[df_analisis_full['Status'] == 'SALAH'])}")
print(f"File CSV disimpan : {nama_file_csv}")

# Tampilkan contoh error dengan confidence tinggi (Overconfident)
errors = df_analisis_full[df_analisis_full['Status'] == 'SALAH']
if len(errors) > 0:
    print("\nContoh 5 Data Salah (Confidence Tertinggi):")
    print(errors.sort_values(by='Confidence', ascending=False).head(5)[['Status', 'Prediksi_MLP', 'Label_Asli', 'Confidence', 'Avg_Monthly_Hours', 'Job_Satisfaction']])
else:
    print("Sempurna! Tidak ada error.")

=== PENGUJIAN DETAIL: Config-3 (70:30) ===

>>> DATA METRIK (Copy ke Sheet 'Config-3 (70:30) Test'):
   No Split Data Configuration        MLP Architecture  Learning Rate Solver  \
0   1                    70:30  (256, 128, 64, 32, 16)         0.1111    sgd   

   Momentum Activation  Loss Test (MSE)  Acc Test  Prec_Class_0  Prec_Class_1  \
0       0.9       relu             0.02      0.98        0.9977        0.9661   

   Prec_Class_2  Prec_Avg  Rec_Class_0  Rec_Class_1  Rec_Class_2  Rec_Avg  \
0         0.956    0.9804       0.9769       0.9771        0.997     0.98   

   F1_Class_0  F1_Class_1  F1_Class_2  F1_Avg  Supp_Class_0  Supp_Class_1  \
0      0.9872      0.9716      0.9761  0.9801        2250.0        1575.0   

   Supp_Class_2  Supp_Total  Random State  Validation Fraction  
0         675.0      4500.0            52                  0.1  

>>> ANALISIS ERROR:
Total Data Test : 4500
Total Salah     : 90
File CSV disimpan : Analisis_Full_4500_Data_Config3.csv

Contoh 5 Data

d:\Pemograman Python\Pengembangan Model MLP\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report

# ==========================================
# KONFIGURASI 3: 70:30 (SGD) - FINAL REPORT
# ==========================================

# 1. LOAD DATA
try:
    df = pd.read_csv('../data/processed_attrition_data.csv')
except FileNotFoundError:
    df = pd.read_csv('processed_attrition_data.csv') # Fallback

X = df.drop('Attrition_Risk_Level', axis=1)
y = df['Attrition_Risk_Level']

# 2. SETTING PARAMETER
config = {
    'name': 'Config-3 (70:30)',
    'test_size': 0.30,  # 30% Data Test
    'params': {
        'hidden_layer_sizes': (256, 128, 64, 32, 16),
        'activation': 'relu',
        'solver': 'sgd',
        'learning_rate_init': 0.1111,
        'max_iter': 18,
        'random_state': 52 
    }
}

print(f"=== PENGUJIAN DETAIL: {config['name']} ===")

# 3. SPLIT DATA
indices = np.arange(len(X))
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=config['test_size'], random_state=42, stratify=y
)

# 4. SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. TRAINING
model = MLPClassifier(**config['params'])
model.fit(X_train_scaled, y_train)

# 6. PREDIKSI
y_pred = model.predict(X_test_scaled)
proba = model.predict_proba(X_test_scaled)

# =========================================================
# BAGIAN A: OUTPUT TABEL METRIK (Untuk Sheet Bagian II)
# =========================================================
report = classification_report(y_test, y_pred, output_dict=True)
acc = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

laporan_dict = {
    'No': [1],
    'Split Data Configuration': ['70:30'],
    'MLP Architecture': [str(config['params']['hidden_layer_sizes'])],
    'Learning Rate': [config['params']['learning_rate_init']],
    'Solver': [config['params']['solver']],
    'Momentum': [0.9],
    'Activation': [config['params']['activation']],
    'Loss Test (MSE)': [round(mse, 5)],
    'Acc Test': [round(acc, 4)],
    
    # --- PRECISION ---
    'Prec_Class_0': [round(report['0']['precision'], 4)],
    'Prec_Class_1': [round(report['1']['precision'], 4)],
    'Prec_Class_2': [round(report['2']['precision'], 4)],
    'Prec_Avg':     [round(report['weighted avg']['precision'], 4)],
    
    # --- RECALL ---
    'Rec_Class_0': [round(report['0']['recall'], 4)],
    'Rec_Class_1': [round(report['1']['recall'], 4)],
    'Rec_Class_2': [round(report['2']['recall'], 4)],
    'Rec_Avg':     [round(report['weighted avg']['recall'], 4)],
    
    # --- F1 SCORE ---
    'F1_Class_0': [round(report['0']['f1-score'], 4)],
    'F1_Class_1': [round(report['1']['f1-score'], 4)],
    'F1_Class_2': [round(report['2']['f1-score'], 4)],
    'F1_Avg':     [round(report['weighted avg']['f1-score'], 4)],
    
    # --- SUPPORT ---
    'Supp_Class_0': [report['0']['support']],
    'Supp_Class_1': [report['1']['support']],
    'Supp_Class_2': [report['2']['support']],
    'Supp_Total':   [report['macro avg']['support']],
    
    'Random State': [config['params']['random_state']],
    'Validation Fraction': [0.1]
}

df_laporan = pd.DataFrame(laporan_dict)

# --- SIMPAN CSV METRIK ---
nama_file_metrik = 'Laporan_Testing_Config3_7030.csv'
df_laporan.to_csv(nama_file_metrik, index=False)

print("\n>>> DATA METRIK (Tersimpan di CSV):")
pd.set_option('display.max_columns', None)
print(df_laporan)
print(f"[INFO] File metrik disimpan: {nama_file_metrik}")

# =========================================================
# BAGIAN B: OUTPUT ANALISIS ERROR (Untuk Sheet Bagian III)
# =========================================================

# Ambil fitur asli data test
df_analisis_full = X.loc[idx_test].copy()

# Masukkan hasil prediksi
df_analisis_full.insert(0, 'Status', np.where(y_test == y_pred, 'Benar', 'SALAH'))
df_analisis_full.insert(1, 'Prediksi_MLP', y_pred)
df_analisis_full.insert(2, 'Label_Asli', y_test.values)
df_analisis_full.insert(3, 'Confidence', np.round(np.max(proba, axis=1), 4))

# --- SIMPAN CSV ANALISIS FULL ---
nama_file_analisis = 'Analisis_Full_4500_Data_Config3.csv'
df_analisis_full.to_csv(nama_file_analisis)

print(f"\n>>> ANALISIS ERROR:")
print(f"Total Data Test : {len(y_test)}")
print(f"Total Salah     : {len(df_analisis_full[df_analisis_full['Status'] == 'SALAH'])}")
print(f"[INFO] File analisis full disimpan: {nama_file_analisis}")

# Tampilkan contoh error dengan confidence tinggi
errors = df_analisis_full[df_analisis_full['Status'] == 'SALAH']
if len(errors) > 0:
    print("\nContoh 5 Data Salah (Confidence Tertinggi):")
    print(errors.sort_values(by='Confidence', ascending=False).head(5)[['Status', 'Prediksi_MLP', 'Label_Asli', 'Confidence', 'Avg_Monthly_Hours', 'Job_Satisfaction']])

=== PENGUJIAN DETAIL: Config-3 (70:30) ===

>>> DATA METRIK (Tersimpan di CSV):
   No Split Data Configuration        MLP Architecture  Learning Rate Solver  \
0   1                    70:30  (256, 128, 64, 32, 16)         0.1111    sgd   

   Momentum Activation  Loss Test (MSE)  Acc Test  Prec_Class_0  Prec_Class_1  \
0       0.9       relu             0.02      0.98        0.9977        0.9661   

   Prec_Class_2  Prec_Avg  Rec_Class_0  Rec_Class_1  Rec_Class_2  Rec_Avg  \
0         0.956    0.9804       0.9769       0.9771        0.997     0.98   

   F1_Class_0  F1_Class_1  F1_Class_2  F1_Avg  Supp_Class_0  Supp_Class_1  \
0      0.9872      0.9716      0.9761  0.9801        2250.0        1575.0   

   Supp_Class_2  Supp_Total  Random State  Validation Fraction  
0         675.0      4500.0            52                  0.1  
[INFO] File metrik disimpan: Laporan_Testing_Config3_7030.csv

>>> ANALISIS ERROR:
Total Data Test : 4500
Total Salah     : 90
[INFO] File analisis full dis

d:\Pemograman Python\Pengembangan Model MLP\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  warnings.warn(
